# Searching or questioning all the sources
In this notebook, we create a chain that can ask all the different sources a question. Using functions we can choose the right source. We use the language model to make the selection.
- Product search: use the VectorStore retriever making use of Amazon OpenSearch Service
- Question: use the VectorStore QA retriever to use content from the vector store and OpenAI to write an asnwer
- Find Opening times: Use a normal query against Amazon OpenSearch Service

## Initialise the OpenSearch connection 
Notice that we do not provide a default alias. We need to provide the name of the index or alias on each request.

In [1]:
import langchain

from retriever import find_auth_opensearch, OpenSearchClient

config = find_auth_opensearch()
client = OpenSearchClient(config)

if client.ping():
    print("We have a connection to the Amazon OpenSearch Cluster")
else:
    print("ERROR: no connection to the Amazon OpenSearch Cluster")

We have a connection to the Amazon OpenSearch Cluster


## Initialise the product search

In [2]:
import os

from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from opensearchpy import RequestsHttpConnection
from dotenv import load_dotenv

load_dotenv()

vector_store_products = OpenSearchVectorSearch(
    index_name="sg-products",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)


def execute_product_search(query: str):
    found_docs = vector_store_products.similarity_search_with_score(query=query, text_field="title",
                                                                    vector_field="title_vector")
    results = []
    for doc, _score in found_docs:
        results.append({"title": doc.page_content, "score": _score})

    return {
        "tool": "product_search",
        "result": results
    }

In [3]:
print(execute_product_search(query="The lord of the rings"))

{'tool': 'product_search', 'result': [{'title': 'Frodo™ & Gollem™', 'score': 0.783552}, {'title': 'Aragorn™ & Arwen™', 'score': 0.7829778}, {'title': 'Gandalf de Grijze™ & Balrog™', 'score': 0.75242954}, {'title': 'Slag om Endor™ helden', 'score': 0.7370903}]}


## Initialise the content search

In [4]:
vector_store = OpenSearchVectorSearch(
    index_name="sg-content",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)

from langchain.chains import RetrievalQA
from langchain import PromptTemplate, OpenAI

prompt_template = """Use the context to answer the question. If you don't know the 
    answer, just say that you don't know, don't make up an answer.

    {context}

    Question: {question}:"""

custom_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": custom_prompt}
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs=chain_type_kwargs
)


def execute_content_search(query: str):
    return {
        "tool": "content_search",
        "result": chain.run(query)
    }

In [5]:
print(execute_content_search("What is your return policy"))

{'tool': 'content_search', 'result': '\n\nOur return policy can be found on our website under the "Returns" section. If you have any additional questions, please contact our customer support team.'}


In [6]:
def execute_opening_hours(query: str):
    body = {
        "query": {
            "match": {
                "city": query
            }
        }
    }
    search_result = client.search(body=body, size=1, index_name="sg-stores")
    result = []
    if search_result["hits"]["total"]["value"] > 0:
        result = search_result["hits"]["hits"][0]["_source"]
        
    
    return {
        "tool": "opening_hours",
        "result": result
    }

In [7]:
print(execute_opening_hours("pijnacker"))

{'tool': 'opening_hours', 'result': {'store_name': 'Trendy Finds', 'city': 'Pijnacker', 'street': 'Kerkweg 1', 'telephone': '06-12345678', 'opening_hours': [{'week_day': 'monday', 'open_time': '11:00', 'closing_time': '18:00'}, {'week_day': 'tuesday', 'open_time': '09:00', 'closing_time': '18:00'}, {'week_day': 'wednesday', 'open_time': '09:00', 'closing_time': '18:00'}, {'week_day': 'thursday', 'open_time': '11:00', 'closing_time': '18:00'}, {'week_day': 'friday', 'open_time': '09:00', 'closing_time': '18:00'}, {'week_day': 'saturday', 'open_time': '10:00', 'closing_time': '17:00'}, {'week_day': 'sunday', 'open_time': '00:00', 'closing_time': '00:00'}]}}


## Initialise the agent using the previous functions

In [13]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor, initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI


def init_agent() -> AgentExecutor:
    llm = ChatOpenAI(model="gpt-4-0613", temperature=0, openai_api_key=os.getenv('OPEN_AI_API_KEY'))
    system_message = SystemMessage(
        content="You are a web service returning the pure json without a change.")
    prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

    tools = [
        Tool(name="ExecuteContentSearch",
             func=execute_content_search,
             description="Useful for obtaining answers to questions about help for the website, your account and sustainability."
             ),
        Tool(name="ExecuteProductSearch",
             func=execute_product_search,
             description="Useful for finding products related to the search terms that are provided."
             ),
        Tool(name="ExecuteOpeningHours",
             func=execute_opening_hours,
             description="Useful for obtaining opening hours of a store in city that is provided."
             ),

    ]

    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)


agent_executor = init_agent()


In [15]:
import langchain
langchain.debug = False
# message = "lord of the rings"
# message = "Do you use reusable packaging"
# message = "How can I reset my password"
# message = "Do you have something from Lord of the rings?"
message = "openinghours pijnacker"
# message = "openingstijden pijnacker"
response = agent_executor.run(message)

print(response)

{
  "tool": "opening_hours",
  "result": {
    "store_name": "Trendy Finds",
    "city": "Pijnacker",
    "street": "Kerkweg 1",
    "telephone": "06-12345678",
    "opening_hours": [
      {
        "week_day": "monday",
        "open_time": "11:00",
        "closing_time": "18:00"
      },
      {
        "week_day": "tuesday",
        "open_time": "09:00",
        "closing_time": "18:00"
      },
      {
        "week_day": "wednesday",
        "open_time": "09:00",
        "closing_time": "18:00"
      },
      {
        "week_day": "thursday",
        "open_time": "11:00",
        "closing_time": "18:00"
      },
      {
        "week_day": "friday",
        "open_time": "09:00",
        "closing_time": "18:00"
      },
      {
        "week_day": "saturday",
        "open_time": "10:00",
        "closing_time": "17:00"
      },
      {
        "week_day": "sunday",
        "open_time": "00:00",
        "closing_time": "00:00"
      }
    ]
  }
}
